In [1]:
import pandas as pd
import numpy as np

In [2]:
%matplotlib inline

In [3]:
df = pd.read_csv("./data/data.csv")

In [4]:
df_y = df['label'].copy()
del df['label']

In [5]:
cols_to_delete = ['18','25','29', '31', '32', '35', '23', '26', '58']
for col in cols_to_delete:
    del df[col]

43